In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_file_df = pd.read_csv(train_file_path,sep='\t',names=['class','contents'])
train_file_df['class'] = train_file_df['class'].map({'ham': 1, 'spam': 0})
train_file_df.head()

In [ ]:
test_file_df = pd.read_csv(test_file_path,sep='\t',names=['class','contents'])
test_file_df['class'] = test_file_df['class'].map({'ham': 1, 'spam': 0})
test_file_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(stop_words='english',max_features=20)

In [ ]:
keywords = cv.fit_transform(train_file_df[train_file_df['class']==0]['contents'])
frequency = zip(cv.get_feature_names_out(), keywords.sum(axis=0).tolist()[0])

sorted_frequency = sorted(frequency, reverse=True)
print(sorted_frequency)

In [ ]:
from sklearn.model_selection import train_test_split

X = train_file_df['contents']
y = train_file_df['class']
Xtest = test_file_df['contents']
ytest = test_file_df['class']
print(y)

In [ ]:
Xt = cv.transform(X)
Xtester = cv.transform(Xtest)



In [ ]:
y_train = np.array(y)
y_test = np.array(ytest)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=Xt.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:

print(Xtester.dtype)
print(y_train.dtype)
print(y_test.dtype)
print(Xt.shape)
print(Xtester.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
model.fit(Xt, y_train, epochs=50, batch_size=32, validation_data=(Xtester, y_test))

In [ ]:

# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(msg):
  print(msg)
  msg_vectorized = cv.transform([msg])
  prediction = model.predict(msg_vectorized.toarray())
  predicted_class = "ham" if prediction[0] >= 0.9 else "spam" # Return the prediction result
  print(prediction[0])
  if predicted_class == "ham":
    print(predicted_class)
    return (1, predicted_class)

  else:
    print(predicted_class)
    return (0, predicted_class)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()